In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


import skimage.io as io
from skimage import util
from skimage.filters import gaussian

import warnings
warnings.filterwarnings('ignore')
from glob import glob



import sys
sys.path.append('../src/')
import image_denosing
import segmentation
import img_diagnostic
import measurements
import tracking
import viz_segmentation_video
import viz_trajectories_to_video
import anchor_points
import viz_one_trajectory

## Preparation 
### Create results folders

In [ ]:
os.mkdir("../results/anchor_area_csv")
os.mkdir("../results/anchor_area_density_plot_pdf")
os.mkdir("../results/anchor_area_density_plot/")
os.mkdir("../results/img_diagnostics")
os.mkdir("../results/video_one_trajectory")
os.mkdir("../results/video_trajectories")
os.mkdir("../results/measurements_csv/")
os.mkdir("../results/tracks_csv/")
os.mkdir("../results/video_segmentation/")

In [ ]:
Diagnostic_images=[]

i=16
file_number=str(i).zfill(3)
file_name="../data/2021nov14_8ml_kb_wt_140X_2sec3min-{}_under_surface.tiff".format(file_number)
print(file_name)

## background removal
images=io.imread(file_name)
images=util.img_as_float(images)
smoothed,background,denoised=image_denosing.run_image_denosing(images)

# write to the data folder
folder_to_save=os.path.split(file_name)[0]
file_prefix=os.path.split(file_name)[1].split('.')[0]
io.imsave(os.path.join(folder_to_save,file_prefix+"_denoised.tif"),denoised)


#     # read
#     folder_to_save=os.path.split(file_name)[0]
#     file_prefix=os.path.split(file_name)[1].split('.')[0]
#     denoised=io.imread(os.path.join(folder_to_save,file_prefix+"_denoised.tif"))


#segmentation
images_binary, threshold=segmentation.outsu_threshold(denoised)
dict_results=img_diagnostic.img_characteristrics(denoised,i,threshold,file_prefix)
io.imsave(os.path.join(folder_to_save,file_prefix+"_segmented.tif"),images_binary) # write to the data folder
Diagnostic_images.append(dict_results)
viz_segmentation_video.segmentation_check_video(images,denoised,images_binary,file_prefix)


# measurements
measurements_df=measurements.props_measurement(images_binary,denoised)
measurements_df=measurements_df[measurements_df["area"]>25] # filter small objects
measurements_df.to_csv(os.path.join("../results/measurements_csv/",file_prefix+"_measurements.csv"))


## tracking
tracks=tracking.tracking(measurements_df,file_prefix)
tracks.to_csv("../results/tracks_csv/{}_Tracks.csv".format(file_prefix))
viz_trajectories_to_video.trajectories_to_video(images_binary,tracks,"../results/video_trajectories/",file_prefix)


## anchor points
## anchor points
anchor_df=anchor_points.plot_anchor_points(images_binary,tracks,file_prefix)
folder_to_save_single_trajectory="../results/video_one_trajectory/"+file_prefix
os.mkdir(folder_to_save_single_trajectory)
for i in anchor_df["particle"].index:
    particle=anchor_df["particle"].loc[i]
    overlap_ratio=anchor_df["overlapped_area_ratio"].loc[i]
    type_object=anchor_df["type"].loc[i]
    traj=tracks[tracks["particle"]==particle]
    viz_one_trajectory.one_trajectory_viz_with_tracks(images_binary,traj,overlap_ratio,folder_to_save_single_trajectory,file_prefix,type_object)

## Test Block

In [ ]:
denoised=io.imread("../data/2021nov14_8ml_kb_wt_140X_2sec3min-016_under_surface_denoised.tif")
images_binary=io.imread("../data/2021nov14_8ml_kb_wt_140X_2sec3min-016_under_surface_segmented.tif")
tracks=pd.read_csv("../results/tracks_csv/2021nov14_8ml_kb_wt_140X_2sec3min-016_under_surface_Tracks.csv")
file_prefix="Experiment_2021nov14_8ml_kb_wt_140X_2sec3min-016_under_surface"
#anchor_df_old=pd.read_csv("../results/Anchor_area_2021nov14_8ml_kb_wt_140X_2sec3min-016_under_surface.csv")

In [ ]:
anchor_df=pd.read_csv("../results/anchor_area_csv/Anchor_area_{}.csv".format(file_prefix))
particle=782
traj=tracks[tracks["particle"]==particle]
overlap_ratio=anchor_df[anchor_df["particle"]==particle]["overlapped_area_ratio"].values[0]
viz_one_trajectory.one_trajectory_viz_with_tracks(images_binary,traj,overlap_ratio,"../results/",file_prefix)